Artilces:<br/>
[Perceptrons]()<br/>
[The Organization of Behavior]()<br/>
[Learning Internal Representations by Error Propagation]()<br/>
[A logical calculus of the ideas immanent in nervous activity]()<br/>
[The perceptron: A probabilistic model for information storage and organization in the brain]()<br/>


